# Chrome Dino re-write 2023

Use Pytorch, stable-baselines3, etc. to build a model to play chrome://dino with reinforcement learning.

## dependencies
- CUDA-enabled Pytorch (Pytorch 2.00; CUDA 1.18)
- Stable-Baselines3 (with extras like OpenCV): https://stable-baselines3.readthedocs.io/en/master/
- Protobuf (a training dependency) (3.20.*)
- pytesseract (interface to Google Tesseract)
- Google Tesseract-OCR ((5.3.1.20230401)
- Gym (gym v0.21 since this is used by Stable-Baselines3; RL environment library): https://gymnasium.farama.org/
- MSS (crossplatform screenshots)
- openCV (2)
- selenium (chrome test driver) (ChromeDriver 112.0.5615.49)

In [ ]:
%%capture
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
%%capture
# needed to support old version of gym (0.21) used with Stable-Baselines3
# gym 0.21 has installation issue; gym moved to Gymnasium; SB3 still does not support Gymnasium 
!pip install setuptools==66 Cmake git+https://github.com/openai/gym.git@9180d12e1b66e7e2a1a622614f787a6ec147ac40

In [ ]:
%%capture
!pip install stable-baselines3[extra] protobuf==3.20.*

In [ ]:
%%capture
!pip install mss pydirectinput pytesseract selenium

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

from mss import mss
from gym import Env
from gym.spaces import Box, Discrete

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

## set up environment

In [ ]:
# create a base Gym environment for managing state
# Env must implement step and reset; optionally, render 
# https://www.gymlibrary.dev/api/core/
class ChromeDinoRL(Env):
    
    # initialize environment spaces
    def __init__(self):
        super().__init__()
        
        # set up observations and actions
        # multidimensional array as return output from observation (an image)
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        # two actions: jump or not
        self.action_space = Discrete(2)
        
        # observation parameters
        self.capture = mss()
        self.game_window = {'top':0, 'left':0, 'width':640, 'height':480}
        self.game_over_window = {'top':1, 'left':1, 'width':1, 'height':1}
        
    # run one timestep in the environment
    def step(self, action):
        # 0 -> spacebar, 1 -> noop
        action_map = {
            0: 'space',
            1: 'no_op'
        }
        
        
    # reset to initial state, return initial observation
    def reset(self):
        pass
        
    # compute render frames for visualization; render with cv2
    # close everything with "q" 
    def render(self):
        cv2.imshow('Chrome Dino', self.current_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()
        
    # do an observation
    def observe_env(self):
        obs = self.capture.grab(self.game_window)
        #obs = np.array(self.capture.grab(self.game_window))[:,:,3].astype(np.uint8)
        # preprocessing 
        
        return obs
        
    # identify game over
    def game_over(self):
        pass
    
    def close(self):
        cv2.destroyAllWindows()
        self.driver.end()
    

## set up driver

automated window setup for consistent environment

In [ ]:
# set up webdriver for selenium on chrome
# requires chromedriver present in path specified
class ChromeDinoDriver():

    def __init__(self):
        self.chrome_driver = None
        self.chrome_path = r"chromedriver\chromedriver.exe" 
        self.startpage = "chrome://dino/"
        self.chrome_options = Options()
        self.chrome_options.add_argument("--window-size=640,480")
        self.driveractive = False
        
    # set up and run driver
    # options 
    def run(self):
        if not self.driveractive:
            self.chrome_driver = webdriver.Chrome(executable_path=self.chrome_path, options=self.chrome_options)
            self.chrome_driver.set_window_position(0, 0, windowHandle='current')
            try:
                self.chrome_driver.get("chrome://dino")
                self.driveractive = True
            except WebDriverException:
                pass # ignore selenium complaining about offline
        else:
            print("Driver already running")

    def end(self): #close driver
        if self.driveractive:
            self.chrome_driver.quit()
            self.driveractive = False
        else:
            print("Driver not running")

### misc testing

In [ ]:
# test environment
env = ChromeDinoRL()

In [ ]:
# random action; observation space empty
print(env.action_space.sample())
plt.imshow(env.observation_space.sample()[0])

In [ ]:
# try an observation
plt.imshow(np.array(env.observe_env()))

In [ ]:
# test driver
driver = ChromeDinoDriver()
driver.run()
time.sleep(2)
driver.end()

In [ ]:
# start driver and try an observation
driver.run()
time.sleep(1)
plt.imshow(np.array(env.observe_env()))
time.sleep(1)
driver.end()

## train model

Train the DQN and save it using a Stable-Baselines3 callback

In [ ]:
# training imports
from stable_baselines3 import DQN
from stablebaselines3.common.callbacks import BaseCallback
import os